In [1]:
mport pandas as pd 
import tensorflow as tf  # Importa o TensorFlow para construção de modelos de aprendizado de máquina
import sklearn
import scikeras

In [115]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.models import Sequential
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import KFold

In [14]:
base = pd.read_csv('autos.csv', encoding ='ISO-8859-1')


In [15]:
# Remove colunas indesejadas do DataFrame 'base' para limpeza de dados.
base = base.drop('dateCrawled', axis=1)    # Remove a coluna 'dateCrawled'.
base = base.drop('dateCreated', axis=1)     # Remove a coluna 'dateCreated'.
base = base.drop('nrOfPictures', axis=1)    # Remove a coluna 'nrOfPictures'.
base = base.drop('postalCode', axis=1)      # Remove a coluna 'postalCode'.
base = base.drop('lastSeen', axis=1)        # Remove a coluna 'lastSeen'.


In [16]:
# Remove colunas adicionais do DataFrame 'base' e filtra os dados com base nos preços.
base = base.drop('name', axis=1)            # Remove a coluna 'name'.
base = base.drop('seller', axis=1)          # Remove a coluna 'seller'.
base = base.drop('offerType', axis=1)       # Remove a coluna 'offerType'.
base = base[base['price'] > 10]              # Filtra o DataFrame para manter apenas os registros com preço maior que 10.
base = base.loc[base['price'] < 350000]     # Filtra o DataFrame para manter apenas os registros com pr


In [17]:
valore = {
    'vehicleType': 'limousine',
    'gearbox': 'manuell',
    'model': 'golf',
    'fuelType': 'benzin',
    'notRepairedDamage': 'nein'
}
base = base.fillna(value = valore)

In [18]:
X = base.iloc[:,1:12].values

In [19]:
y = base.iloc[:,0].values

In [55]:
# Cria um transformador que aplica one-hot encoding a colunas específicas do DataFrame.
onehotencoder = ColumnTransformer(
    transformers=[('Onehot', OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])],  # Aplica OneHotEncoder às colunas nos índices especificados.
    remainder='passthrough'  # Mantém as colunas que não foram transformadas sem alteração.
)


In [76]:
X = onehotencoder.fit_transform(X).toarray()

In [77]:
X.shape

(359291, 316)

In [101]:
# Função para criar o modelo com parâmetros configuráveis, incluindo a função de perda (loss)
def criar_rede(loss_function='mean_squared_error', optimizer='adam', activation='relu', neurons=158):
    """
    Cria uma rede neural sequencial para regressão com parâmetros configuráveis.
    
    Args:
        loss_function: Função de perda a ser utilizada na compilação do modelo.
        optimizer: Otimizador a ser utilizado durante o treinamento do modelo.
        activation: Função de ativação a ser utilizada nas camadas ocultas.
        neurons (int): Número de unidades nas camadas ocultas.

    Returns:
        Sequential: A rede neural configurada.
    """
    tf.keras.backend.clear_session()
    
    model = Sequential([
        tf.keras.layers.InputLayer(shape=(316,)),
        tf.keras.layers.Dense(units=neurons, activation=activation),
        tf.keras.layers.Dense(units=neurons, activation=activation),
        tf.keras.layers.Dense(units=1, activation='linear')
    ])
    
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['mean_squared_error'])
    return model

In [110]:
# Parâmetros de tuning
parametros = {
    'batch_size': [300],
    'epochs': [10],
    'model__optimizer': ['adam'],
    'model__loss_function': ['mean_squared_error', 'mean_absolute_error', 'huber_loss', 'mean_squared_logarithmic_error'],
    'model__activation': ['relu'],
    'model__neurons': [158]
}

In [111]:
# Instância de KerasRegressor para uso com o GridSearchCV
rede_neural = KerasClassifier(model=criar_rede)

In [119]:
# Configuração do GridSearchCV para otimizar a função de perda
kf = KFold(n_splits=2, shuffle=True, random_state=42)  # Use 2 splits instead of 3
grid_search = GridSearchCV(estimator=rede_neural, param_grid=parametros,
                           scoring='neg_mean_squared_error', cv=kf)

In [120]:
# Execução do GridSearchCV (substitua X e y pelo seu conjunto de dados)
grid_search.fit(X, y)

Epoch 1/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1083229.5000 - mean_squared_error: 1083229.5000
Epoch 2/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 468299.4062 - mean_squared_error: 468299.4062
Epoch 3/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 399041.0938 - mean_squared_error: 399041.0938
Epoch 4/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 375211.3438 - mean_squared_error: 375211.3750
Epoch 5/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 334787.2500 - mean_squared_error: 334787.2500
Epoch 6/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 348719.1875 - mean_squared_error: 348719.1875
Epoch 7/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 312715.1875 - mean_squared_error: 312715.1875
Epoch 8/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 304210.6875 - mean_squared_error: 304210.6875
Epoch 9/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 296766.0938 - mean_squared_error: 296766.0938
Epoch 10/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 1s 

C:\Users\jonin\miniconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
2 fits failed out of a total of 8.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\jonin\miniconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jonin\miniconda3\Lib\site-packages\scikeras\wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "C:\Users\jonin\miniconda3\Lib\site-packages\scikeras\wrappers.py", line 770, in fit
    self._fit(
  File "C:\Users\jonin\miniconda3\Lib\site-pac

Epoch 1/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4043690.2500 - mean_squared_error: 4043690.2500
Epoch 2/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 601240.1875 - mean_squared_error: 601240.1875
Epoch 3/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 586868.1250 - mean_squared_error: 586868.1250
Epoch 4/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 557499.6875 - mean_squared_error: 557499.6875
Epoch 5/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 523260.2812 - mean_squared_error: 523260.2812
Epoch 6/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 492657.0625 - mean_squared_error: 492657.0625
Epoch 7/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 461252.9688 - mean_squared_error: 461252.9688
Epoch 8/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 435372.6562 - mean_squared_error: 435372.6562
Epoch 9/10
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 427937.7188 - mean_squared_error: 427937.7188
Epoch 10/10
1198/1198 ━━━━

GridSearchCV(cv=KFold(n_splits=2, random_state=42, shuffle=True),
             estimator=KerasClassifier(model=<function criar_rede at 0x000001BE3F726C00>),
             param_grid={'batch_size': [300], 'epochs': [10],
                         'model__activation': ['relu'],
                         'model__loss_function': ['mean_squared_error',
                                                  'mean_absolute_error',
                                                  'huber_loss',
                                                  'mean_squared_logarithmic_error'],
                         'model__neurons': [158],
                         'model__optimizer': ['adam']},
             scoring='neg_mean_squared_error')

In [121]:
# Resultado do melhor parâmetro encontrado
# Imprime a melhor função de perda encontrada pelo GridSearch
print("Melhor função de perda:", grid_search.best_params_['model__loss_function'])
# Imprime o melhor MSE encontrado (inverter o sinal porque o GridSearchCV retorna valores negativos)
print("Melhor MSE:", -grid_search.best_score_)


Melhor função de perda: mean_squared_error
Melhor MSE: 112336815.04998016
